In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


# Tableone without package

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle

In [5]:
dataset = pd.read_csv('/content/drive/MyDrive/Research on AI Medicine/Perdicting mortality/Data/dataset_2/dataset.csv')

In [ ]:
dataset.info()

In [8]:
dataset = dataset.drop(columns=['hadm_id'])

In [9]:
dataset['Length of ICU stay'] = dataset['Length of ICU stay']/ (24*60*60)

In [10]:
dataset['race'] = dataset['race'].str.replace(r'.*AFRICAN AMERICAN.*', 'African american', regex=True)
dataset['race'] = dataset['race'].str.replace(r'.*WHITE.*', 'White', regex=True)
dataset['race'] = dataset['race'].str.replace(r'.*HISPANIC.*', 'Hispanic', regex=True)
dataset.loc[~dataset['race'].isin(['White', 'African american', 'Hispanic']), 'race'] = 'Other'
dataset['gender'] = dataset['gender'].str.replace(r'.*M.*', 'Male', regex=True)
dataset['gender'] = dataset['gender'].str.replace(r'.*F.*', 'Female', regex=True)

In [26]:
# eliminate outliers
dataset_copy = dataset.copy()
predictors_continuous = dataset_copy

predictors_continuous['Maximum creatinine'] = predictors_continuous['Maximum creatinine'].where(predictors_continuous['Maximum creatinine'] <= 50, np.nan)
predictors_continuous['Minimum creatinine'] = predictors_continuous['Minimum creatinine'].where(predictors_continuous['Minimum creatinine'] <= 50, np.nan)
predictors_continuous['Maximum creatinine during day2 and day3'] = predictors_continuous['Maximum creatinine during day2 and day3'].where(predictors_continuous['Maximum creatinine during day2 and day3'] <= 1000, np.nan)

predictors_continuous['Maximum heart rate'] = predictors_continuous['Maximum heart rate'].where(predictors_continuous['Maximum heart rate'] <= 300, np.nan)
predictors_continuous['Mean heart rate'] = predictors_continuous['Mean heart rate'].where(predictors_continuous['Mean heart rate'] <= 300, np.nan)

predictors_continuous['Minimum spo2'] = predictors_continuous['Minimum spo2'].where(predictors_continuous['Minimum spo2'] <= 100, np.nan)
predictors_continuous['Mean spo2'] = predictors_continuous['Mean spo2'].where(predictors_continuous['Mean spo2'] <= 100, np.nan)

predictors_continuous['Minimum prothrombin'] = predictors_continuous['Minimum prothrombin'].where(predictors_continuous['Minimum prothrombin'] <= 1000, np.nan)
predictors_continuous['Maximum prothrombin'] = predictors_continuous['Maximum prothrombin'].where(predictors_continuous['Maximum prothrombin'] <= 1000, np.nan)

predictors_continuous['Mean systolic bp'] = predictors_continuous['Mean systolic bp'].where(predictors_continuous['Mean systolic bp'] <= 500, np.nan)

predictors_continuous['Mean diastolic bp'] = predictors_continuous['Mean diastolic bp'].where(predictors_continuous['Mean diastolic bp'] <= 2000, np.nan)

predictors_continuous['Maximum respiratory rate'] = predictors_continuous['Maximum respiratory rate'].where(predictors_continuous['Maximum respiratory rate'] <= 500, np.nan)

predictors_continuous['Mean respiratory rate'] = predictors_continuous['Mean respiratory rate'].where(predictors_continuous['Mean respiratory rate'] <= 500, np.nan)

predictors_continuous['Maximum sodium'] = predictors_continuous['Maximum sodium'].where(predictors_continuous['Maximum sodium'] <= 500, np.nan)

# predictors_continuous['Sodium Mean'].plot()
# predictors_continuous.info()

In [27]:
predictors_continuous.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30096 entries, 0 to 30095
Data columns (total 38 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   stay_id                                  30096 non-null  int64  
 1   Age                                      30096 non-null  int64  
 2   gender                                   30096 non-null  object 
 3   race                                     30096 non-null  object 
 4   mortality                                30096 non-null  int64  
 5   Minimum creatinine                       29865 non-null  float64
 6   Maximum creatinine                       29861 non-null  float64
 7   Maximum creatinine during day2 and day3  29788 non-null  float64
 8   Maximum heart rate                       29950 non-null  float64
 9   Mean heart rate                          30009 non-null  float64
 10  Minimum systolic bp                      26528

In [28]:
mortality_1 = predictors_continuous[predictors_continuous['mortality']==1]
mortality_1_continuous = mortality_1.drop(columns=['gender', 'race'])

In [29]:
mortality_1_continuous.mean(axis=0, skipna=True)

stay_id                                    3.498480e+07
Age                                        6.955988e+01
mortality                                  1.000000e+00
Minimum creatinine                         1.717604e+00
Maximum creatinine                         1.970049e+00
Maximum creatinine during day2 and day3    1.993802e+00
Maximum heart rate                         1.291188e+02
Mean heart rate                            8.748641e+01
Minimum systolic bp                        9.049952e+01
Mean systolic bp                           1.153166e+02
Minimum diastolic bp                       4.526471e+01
Mean diastolic bp                          6.235794e+01
Minimum spo2                               8.549222e+01
Mean spo2                                  8.605205e+01
Minimum hemoglobin                         9.530325e+00
Minimum prothrombin                        1.701074e+01
Maximum prothrombin                        1.933791e+01
Minimum respiratory rate                   1.232

In [30]:
mortality_1_continuous.std(axis=0, skipna=True)

stay_id                                    2.880079e+06
Age                                        1.394583e+01
mortality                                  0.000000e+00
Minimum creatinine                         1.612770e+00
Maximum creatinine                         1.877195e+00
Maximum creatinine during day2 and day3    1.828490e+00
Maximum heart rate                         1.219906e+01
Mean heart rate                            1.402870e+01
Minimum systolic bp                        1.865984e+01
Mean systolic bp                           1.823578e+01
Minimum diastolic bp                       1.242447e+01
Mean diastolic bp                          1.454726e+01
Minimum spo2                               3.043272e+00
Mean spo2                                  1.645716e+00
Minimum hemoglobin                         2.126997e+00
Minimum prothrombin                        8.412891e+00
Maximum prothrombin                        1.286347e+01
Minimum respiratory rate                   4.727

In [33]:
mortality_1_categorical = mortality_1[['gender', 'race']]

In [43]:
# mortality_1_categorical['gender'].value_counts()
# mortality_1_categorical['gender'].value_counts() / mortality_1_categorical['gender'].count()
mortality_1_categorical['race'].value_counts()
mortality_1_categorical['race'].value_counts() / mortality_1_categorical['race'].count()

race
White               0.680740
Other               0.180997
African american    0.109039
Hispanic            0.029224
Name: count, dtype: float64

In [44]:
mortality_0 = predictors_continuous[predictors_continuous['mortality']==0]
mortality_0_continuous = mortality_0.drop(columns=['gender', 'race'])

In [46]:
mortality_0_continuous.mean(axis=0, skipna=True)

stay_id                                    3.497633e+07
Age                                        6.342411e+01
mortality                                  0.000000e+00
Minimum creatinine                         1.261476e+00
Maximum creatinine                         1.464353e+00
Maximum creatinine during day2 and day3    1.449306e+00
Maximum heart rate                         1.274289e+02
Mean heart rate                            8.605507e+01
Minimum systolic bp                        9.614454e+01
Mean systolic bp                           1.186554e+02
Minimum diastolic bp                       5.001436e+01
Mean diastolic bp                          6.575097e+01
Minimum spo2                               8.553900e+01
Mean spo2                                  8.607472e+01
Minimum hemoglobin                         1.007842e+01
Minimum prothrombin                        1.461939e+01
Maximum prothrombin                        1.649953e+01
Minimum respiratory rate                   1.155

In [47]:
mortality_0_continuous.std(axis=0, skipna=True)

stay_id                                    2.898071e+06
Age                                        1.534909e+01
mortality                                  0.000000e+00
Minimum creatinine                         1.363622e+00
Maximum creatinine                         1.619351e+00
Maximum creatinine during day2 and day3    1.531203e+00
Maximum heart rate                         1.214771e+01
Mean heart rate                            1.364342e+01
Minimum systolic bp                        1.884526e+01
Mean systolic bp                           1.849783e+01
Minimum diastolic bp                       1.276608e+01
Mean diastolic bp                          1.451482e+01
Minimum spo2                               3.474345e+00
Mean spo2                                  1.844813e+00
Minimum hemoglobin                         2.292644e+00
Minimum prothrombin                        5.709461e+00
Maximum prothrombin                        9.118238e+00
Minimum respiratory rate                   4.614

In [50]:
mortality_0_categorical = mortality_0[['gender', 'race']]

In [54]:
mortality_0_categorical['gender'].value_counts()
mortality_0_categorical['gender'].value_counts() / mortality_0_categorical['gender'].count()
mortality_0_categorical['race'].value_counts()
mortality_0_categorical['race'].value_counts() / mortality_0_categorical['race'].count()

race
White               0.671117
Other               0.209990
African american    0.080051
Hispanic            0.038842
Name: count, dtype: float64

In [62]:
import scipy.stats as stats

# Sample data: results from two groups
data_group1 = [23, 20, 19, 22, 25, 18, 24, 20, 19, 22]
data_group2 = [28, 30, 29, 26, 28, 33, 25, 26, 30, 33]

# Perform a t-test on two independent samples
t_stat, p_value = stats.ttest_ind(data_group1, data_group2)

print("T-statistic:", t_stat)
print("P-value:", p_value)

T-statistic: -6.603842688543812
P-value: 3.3517364727377473e-06


# Tableone Package

In [ ]:
!pip install tableone

In [9]:
from tableone import TableOne, load_dataset
import pandas as pd
import numpy as np

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
dataset = pd.read_csv('/content/drive/MyDrive/Research on AI Medicine/Perdicting mortality/Data/dataset_2/dataset.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
dataset = dataset.drop(columns=['hadm_id'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
dataset['Length of ICU stay'] = dataset['Length of ICU stay']/ (24*60*60)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
dataset['race'] = dataset['race'].str.replace(r'.*AFRICAN AMERICAN.*', 'African american', regex=True)
dataset['race'] = dataset['race'].str.replace(r'.*WHITE.*', 'White', regex=True)
dataset['race'] = dataset['race'].str.replace(r'.*HISPANIC.*', 'Hispanic', regex=True)
dataset.loc[~dataset['race'].isin(['White', 'African american', 'Hispanic']), 'race'] = 'Other'
dataset['gender'] = dataset['gender'].str.replace(r'.*M.*', 'Male', regex=True)
dataset['gender'] = dataset['gender'].str.replace(r'.*F.*', 'Female', regex=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30096 entries, 0 to 30095
Data columns (total 38 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   stay_id                                  30096 non-null  int64  
 1   Age                                      30096 non-null  int64  
 2   gender                                   30096 non-null  object 
 3   race                                     30096 non-null  object 
 4   mortality                                30096 non-null  int64  
 5   Minimum creatinine                       29865 non-null  float64
 6   Maximum creatinine                       29865 non-null  float64
 7   Maximum creatinine during day2 and day3  29792 non-null  float64
 8   Maximum heart rate                       30035 non-null  float64
 9   Mean heart rate                          30035 non-null  float64
 10  Minimum systolic bp                      26528

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# eliminate outliers
dataset_copy = dataset.copy()
predictors_continuous = dataset_copy

predictors_continuous['Maximum creatinine'] = predictors_continuous['Maximum creatinine'].where(predictors_continuous['Maximum creatinine'] <= 50, np.nan)
predictors_continuous['Minimum creatinine'] = predictors_continuous['Minimum creatinine'].where(predictors_continuous['Minimum creatinine'] <= 50, np.nan)
predictors_continuous['Maximum creatinine during day2 and day3'] = predictors_continuous['Maximum creatinine during day2 and day3'].where(predictors_continuous['Maximum creatinine during day2 and day3'] <= 1000, np.nan)

predictors_continuous['Maximum heart rate'] = predictors_continuous['Maximum heart rate'].where(predictors_continuous['Maximum heart rate'] <= 300, np.nan)
predictors_continuous['Mean heart rate'] = predictors_continuous['Mean heart rate'].where(predictors_continuous['Mean heart rate'] <= 300, np.nan)

predictors_continuous['Minimum spo2'] = predictors_continuous['Minimum spo2'].where(predictors_continuous['Minimum spo2'] <= 100, np.nan)
predictors_continuous['Mean spo2'] = predictors_continuous['Mean spo2'].where(predictors_continuous['Mean spo2'] <= 100, np.nan)

predictors_continuous['Minimum prothrombin'] = predictors_continuous['Minimum prothrombin'].where(predictors_continuous['Minimum prothrombin'] <= 1000, np.nan)
predictors_continuous['Maximum prothrombin'] = predictors_continuous['Maximum prothrombin'].where(predictors_continuous['Maximum prothrombin'] <= 1000, np.nan)

predictors_continuous['Mean systolic bp'] = predictors_continuous['Mean systolic bp'].where(predictors_continuous['Mean systolic bp'] <= 500, np.nan)

predictors_continuous['Mean diastolic bp'] = predictors_continuous['Mean diastolic bp'].where(predictors_continuous['Mean diastolic bp'] <= 2000, np.nan)

predictors_continuous['Maximum respiratory rate'] = predictors_continuous['Maximum respiratory rate'].where(predictors_continuous['Maximum respiratory rate'] <= 500, np.nan)

predictors_continuous['Mean respiratory rate'] = predictors_continuous['Mean respiratory rate'].where(predictors_continuous['Mean respiratory rate'] <= 500, np.nan)

predictors_continuous['Maximum sodium'] = predictors_continuous['Maximum sodium'].where(predictors_continuous['Maximum sodium'] <= 500, np.nan)

# predictors_continuous['Sodium Mean'].plot()
# predictors_continuous.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
data = predictors_continuous.copy()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
categorical = ['gender', 'race']
groupby = ['mortality']
nonnormal = ['Age']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
mytable = TableOne(data, categorical=categorical, groupby=groupby, nonnormal=nonnormal, pval=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
pd.set_option('display.float_format', '{:.2f}'.format)
print(mytable.tabulate(tablefmt = "fancy_grid"))

╒════════════════════════════════════════════════════╤══════════════════╤═══════════╤════════════════════════╤════════════════════════╤════════════════════════╤═══════════╕
│                                                    │                  │ Missing   │ Overall                │ 0                      │ 1                      │ P-Value   │
╞════════════════════════════════════════════════════╪══════════════════╪═══════════╪════════════════════════╪════════════════════════╪════════════════════════╪═══════════╡
│ n                                                  │                  │           │ 30096                  │ 16477                  │ 13619                  │           │
├────────────────────────────────────────────────────┼──────────────────┼───────────┼────────────────────────┼────────────────────────┼────────────────────────┼───────────┤
│ stay_id, mean (SD)                                 │                  │ 0         │ 34980164.3 (2889898.3) │ 34976329.8 (2898071.0) │

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
print(mytable.tabulate(tablefmt = "text"))

                                                                      Missing    Overall                 0                       1                       P-Value
--------------------------------------------------  ----------------  ---------  ----------------------  ----------------------  ----------------------  ---------
n                                                                                30096                   16477                   13619
stay_id, mean (SD)                                                    0          34980164.3 (2889898.3)  34976329.8 (2898071.0)  34984803.5 (2880079.1)  0.800
Age, median [Q1,Q3]                                                   0          67.0 [57.0,78.0]        65.0 [54.0,75.0]        71.0 [61.0,80.0]        <0.001
gender, n (%)                                       Female            0          13111 (43.6)            7126 (43.2)             5985 (43.9)             0.229
                                                    Male       

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
